In [38]:
import json
import urllib
import zipfile
import io
import numpy as np

# Load dataset
SW_DATASET_URL = 'https://github.com/SeijiEmery/shape-net-data/raw/master/car-vertex-data/shrinkwrap-data.json.zip'
with urllib.request.urlopen(SW_DATASET_URL) as f:
    with zipfile.ZipFile(io.BytesIO(f.read()), 'r') as zf:
        with zf.open('shrinkwrap-data.json', 'r') as f:
            sw_dataset = json.loads(f.read())
            
sw_keys = list(sw_dataset.keys())
sw_dimensions = set(map(len, sw_dataset.values()))
print("dataset members have the following lengths: %s (expected 294)"%(sw_dimensions,))

sw_keys = [ key for key in sw_dataset.keys() if len(sw_dataset[key]) == 294 ]
non_corrupt_values = [ x for x in sw_dataset.values() if len(x) == 294 ]
corrupt_values = [ x for x in sw_dataset.values() if len(x) != 294 ]
print("Has %s corrupt value(s), %s values, %s keys"%(
    len(corrupt_values), len(non_corrupt_values), len(sw_keys)))

sw_data = np.array(non_corrupt_values)
print("dataset size: %s"%(sw_data.shape,))

x_train, x_test = np.split(sw_data, [500], 0)
print("partitioning as train = %s, test = %s"%(x_train.shape, x_test.shape))

dataset members have the following lengths: {1158, 294} (expected 294)
Has 1 corrupt value(s), 601 values, 601 keys
dataset size: (601, 294)
partitioning as train = (500, 294), test = (101, 294)


In [44]:
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
from keras.models import Model, load_model
import os

INPUT_DIM = 294
HIDDEN_DIM = 80
LATENT_DIM = 10

model = Sequential([
    Dense(HIDDEN_DIM, input_shape=(INPUT_DIM,)),
    Activation('relu'),
    Dense(LATENT_DIM),
    Activation('relu'),
    Dense(HIDDEN_DIM),
    Activation('relu'),
    Dense(INPUT_DIM),
    Activation('linear')
])
trained_epochs = 0
model.compile(optimizer='adam', loss='mse')
model.summary()

def train (epochs, batch_size=50):
    global model, trained_epochs
    print("Training: epoch %s -> %s"%(trained_epochs, epochs + trained_epochs))
    model.fit(x_train, x_train,
             epochs=epochs,
             batch_size=batch_size)
    trained_epochs += epochs
    if not os.path.exists('altmodel'):
        os.makedirs('altmodel')
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(trained_epochs,))
    print("Saving snapshot as %s"%path)
    model.save(path)
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 80)                23600     
_________________________________________________________________
activation_21 (Activation)   (None, 80)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                810       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 80)                880       
_________________________________________________________________
activation_23 (Activation)   (None, 80)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 294)               23814     
__________

In [47]:
for i in range(30):
    train(10)

Training: epoch 210 -> 220
Epoch 1/10
500/500 [==============================] - 0s 48us/step - loss: 9.5483e-04
Epoch 2/10
500/500 [==============================] - 0s 57us/step - loss: 9.5620e-04
Epoch 3/10
500/500 [==============================] - 0s 64us/step - loss: 9.5454e-04
Epoch 4/10
500/500 [==============================] - 0s 50us/step - loss: 9.5621e-04
Epoch 5/10
500/500 [==============================] - 0s 60us/step - loss: 9.5354e-04
Epoch 6/10
500/500 [==============================] - 0s 68us/step - loss: 9.5418e-04
Epoch 7/10
500/500 [==============================] - 0s 70us/step - loss: 9.5680e-04
Epoch 8/10
500/500 [==============================] - 0s 73us/step - loss: 9.5606e-04
Epoch 9/10
500/500 [==============================] - 0s 67us/step - loss: 9.5411e-04
Epoch 10/10
500/500 [==============================] - 0s 72us/step - loss: 9.5281e-04
Saving snapshot as altmodel/altmodel-epoch-220.h5
Training: epoch 220 -> 230
Epoch 1/10
500/500 [===============

500/500 [==============================] - 0s 52us/step - loss: 9.3347e-04
Epoch 9/10
500/500 [==============================] - 0s 56us/step - loss: 9.2886e-04
Epoch 10/10
500/500 [==============================] - 0s 49us/step - loss: 9.2795e-04
Saving snapshot as altmodel/altmodel-epoch-300.h5


In [81]:
def load_model_from_epoch (epoch):
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(epoch,))
    print("Loading %s"%path)
    return load_model(path)

def predict (epoch):
    model = load_model_from_epoch(epoch)
    print("Generating prediction")
    return map(model.predict, (x_train, x_test))
y_train, y_test = predict(200)
y_train.shape, y_test.shape

Loading altmodel/altmodel-epoch-200.h5
Generating prediction


((500, 294), (101, 294))

In [82]:
OBJ_TEMPLATE = '''
# {name}
{vertices}
f 1//1 5//1 6//1 2//1
f 2//2 6//2 58//2 59//2
f 5//3 56//3 57//3 6//3
f 6//4 57//4 91//4 58//4
f 1//5 2//5 7//5 3//5
f 3//6 7//6 61//6 62//6
f 2//7 59//7 60//7 7//7
f 7//8 60//8 92//8 61//8
f 1//9 3//9 8//9 4//9
f 4//10 8//10 64//10 65//10
f 3//11 62//11 63//11 8//11
f 8//12 63//12 94//12 64//12
f 1//13 4//13 9//13 5//13
f 5//14 9//14 55//14 56//14
f 4//15 65//15 66//15 9//15
f 9//16 66//16 93//16 55//16
f 10//17 14//17 15//17 11//17
f 11//18 15//18 66//18 65//18
f 14//19 68//19 69//19 15//19
f 15//20 69//20 93//20 66//20
f 10//21 11//21 16//21 12//21
f 12//22 16//22 70//22 71//22
f 11//23 65//23 64//23 16//23
f 16//24 64//24 94//24 70//24
f 10//25 12//25 17//25 13//25
f 13//26 17//26 73//26 74//26
f 12//27 71//27 72//27 17//27
f 17//28 72//28 98//28 73//28
f 10//29 13//29 18//29 14//29
f 14//30 18//30 67//30 68//30
f 13//31 74//31 75//31 18//31
f 18//32 75//32 97//32 67//32
f 19//33 23//33 24//33 20//33
f 20//34 24//34 75//34 74//34
f 23//35 77//35 78//35 24//35
f 24//36 78//36 97//36 75//36
f 19//37 20//37 25//37 21//37
f 21//38 25//38 79//38 80//38
f 20//39 74//39 73//39 25//39
f 25//40 73//40 98//40 79//40
f 19//41 21//41 26//41 22//41
f 22//42 26//42 82//42 83//42
f 21//43 80//43 81//43 26//43
f 26//44 81//44 96//44 82//44
f 19//45 22//45 27//45 23//45
f 23//46 27//46 76//46 77//46
f 22//47 83//47 84//47 27//47
f 27//48 84//48 95//48 76//48
f 28//49 32//49 33//49 29//49
f 29//50 33//50 84//50 83//50
f 32//51 86//51 87//51 33//51
f 33//52 87//52 95//52 84//52
f 28//53 29//53 34//53 30//53
f 30//54 34//54 88//54 89//54
f 29//55 83//55 82//55 34//55
f 34//56 82//56 96//56 88//56
f 28//57 30//57 35//57 31//57
f 31//58 35//58 60//58 59//58
f 30//59 89//59 90//59 35//59
f 35//60 90//60 92//60 60//60
f 28//61 31//61 36//61 32//61
f 32//62 36//62 85//62 86//62
f 31//63 59//63 58//63 36//63
f 36//64 58//64 91//64 85//64
f 37//65 41//65 42//65 38//65
f 38//66 42//66 69//66 68//66
f 41//67 56//67 55//67 42//67
f 42//68 55//68 93//68 69//68
f 37//69 38//69 43//69 39//69
f 39//70 43//70 78//70 77//70
f 38//71 68//71 67//71 43//71
f 43//72 67//72 97//72 78//72
f 37//73 39//73 44//73 40//73
f 40//74 44//74 87//74 86//74
f 39//75 77//75 76//75 44//75
f 44//76 76//76 95//76 87//76
f 37//77 40//77 45//77 41//77
f 41//78 45//78 57//78 56//78
f 40//79 86//79 85//79 45//79
f 45//80 85//80 91//80 57//80
f 46//81 50//81 51//81 47//81
f 47//82 51//82 72//82 71//82
f 50//83 80//83 79//83 51//83
f 51//84 79//84 98//84 72//84
f 46//85 47//85 52//85 48//85
f 48//86 52//86 63//86 62//86
f 47//87 71//87 70//87 52//87
f 52//88 70//88 94//88 63//88
f 46//89 48//89 53//89 49//89
f 49//90 53//90 90//90 89//90
f 48//91 62//91 61//91 53//91
f 53//92 61//92 92//92 90//92
f 46//93 49//93 54//93 50//93
f 50//94 54//94 81//94 80//94
f 49//95 89//95 88//95 54//95
f 54//96 88//96 96//96 81//96
'''.strip()

def make_obj (verts, name):
    return OBJ_TEMPLATE.format(name=name, vertices='\n'.join([
        'v %s %s %s'%(verts[i], verts[i+1], verts[i+2])
        for i in range(0, len(verts) - 2, 3)
    ]))
print(make_obj(x_train[12], sw_keys[12]))

# car-params/02958343-80b5df2ecf1051fd61c6c61410fc904b.json
v -0.191121 0.0 0.0
v -0.191121 0.006916 0.368814
v -0.16805 0.146578 -0.0
v -0.191121 0.005511 -0.363999
v -0.178162 -0.0654 0.0
v -0.184984 -0.082123 0.304755
v -0.184742 0.092359 0.384368
v -0.185406 0.081136 -0.326389
v -0.184984 -0.082123 -0.297115
v -0.0 0.031295 -0.41837
v -0.191121 0.031295 -0.41837
v -0.0 0.067398 -0.414911
v 0.182494 0.031295 -0.41837
v 0.0 -0.0654 -0.398229
v -0.178164 -0.0654 -0.398229
v -0.191121 0.067398 -0.414911
v 0.182494 0.067398 -0.414911
v 0.168471 -0.0654 -0.398229
v 0.182494 0.0 0.0
v 0.183668 -0.001021 -0.347688
v 0.159942 0.146578 -0.0
v 0.183668 0.001968 0.356492
v 0.168471 -0.0654 0.0
v 0.183339 -0.082123 -0.297674
v 0.177305 0.081136 -0.326389
v 0.175873 0.086533 0.323425
v 0.183339 -0.082123 0.307491
v -0.0 0.012726 0.47663
v 0.182494 0.012726 0.47663
v 0.0 0.082733 0.47663
v -0.191121 0.012726 0.47663
v 0.0 -0.067791 0.455745
v 0.167312 -0.067791 0.455745
v 0.176293 0.092359 0.4766

In [ ]:
import multiprocessing as mp
pool = mp.Pool(12)

def write_obj (args):
    verts, path = args
    print("writing %s"%path)
    with open(path, 'w') as f:
        f.write(make_obj(verts, path))
        
def build_dirs (paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def export_model_objs (epoch, num_samples=10, output_dir='altencoder-objs', parallel=False):
    model = load_model_from_epoch(epoch)
    print("predicting for epoch %s"%epoch)
    y_train, y_test = map(model.predict, (x_train, x_test))
    tasks = []
    print("generating write-obj jobs for epoch %s"%epoch)
    for i in range(num_samples):
        train_name = sw_keys[i].split('/')[1].split('.')[0]
        test_name  = sw_keys[i+500].split('/')[1].split('.')[0]
        
        build_dirs([
            os.path.join(output_dir, str(epoch), 'input'),
            os.path.join(output_dir, str(epoch), 'output'),
        ])
        tasks.append((x_train[i], os.path.join(output_dir, str(epoch), 'input', 'train-%s-input.obj'%train_name)))
        tasks.append((y_train[i], os.path.join(output_dir, str(epoch), 'output', 'train-%s-output.obj'%train_name)))
        tasks.append((x_test[i], os.path.join(output_dir, str(epoch), 'input', 'test-%s-input.obj'%test_name)))
        tasks.append((y_test[i], os.path.join(output_dir, str(epoch), 'output', 'test-%s-output.obj'%test_name)))
    
    print("dispatching write-obj jobs for epoch %s"%epoch)
    if parallel:
        map(write_obj, tasks)
    else:
        pool.map(write_obj, tasks)
    print("done processing models for epoch %s"%epoch)

export_model_objs(300)
export_model_objs(200)
export_model_objs(100)
export_model_objs(50)
export_model_objs(10)
export_model_objs(20)

Loading altmodel/altmodel-epoch-300.h5
predicting for epoch 300
writing altencoder-objs/300/input/train-02958343-835d8e35119c714bfdb3ae79bdfd4e2b-input.obj
writing altencoder-objs/300/output/train-02958343-2ab27beb8a37ba37ac00fd1150223027-output.obj
writing altencoder-objs/300/output/test-02958343-e27019b4147f868dbda733a39f84326d-output.obj
writing altencoder-objs/300/input/train-02958343-2ab27beb8a37ba37ac00fd1150223027-input.obj
writing altencoder-objs/300/input/test-02958343-e27019b4147f868dbda733a39f84326d-input.obj
writing altencoder-objs/300/input/test-02958343-928a5f0e95e7aae5780bcccc86c008c3-input.obj
writing altencoder-objs/300/output/train-02958343-835d8e35119c714bfdb3ae79bdfd4e2b-output.obj
writing altencoder-objs/300/input/train-02958343-d07c8208ad9dc35780e74058ee862f05-input.obj
writing altencoder-objs/300/output/test-02958343-928a5f0e95e7aae5780bcccc86c008c3-output.obj
writing altencoder-objs/300/output/train-02958343-d07c8208ad9dc35780e74058ee862f05-output.obj
writing al

In [48]:
y_train, y_test = model.predict(x_train), model.predict(x_test)

In [49]:
x_train[0]

array([-0.188041,  0.071757, -0.041727, -0.191469,  0.008376,  0.29926 ,
       -0.186761,  0.076401, -0.039019, -0.191926,  0.010706, -0.317276,
       -0.176773, -0.078195, -0.001716, -0.1874  , -0.107019,  0.266355,
       -0.19355 ,  0.027168,  0.255872, -0.193178,  0.020274, -0.28742 ,
       -0.1874  , -0.118797, -0.285778,  0.002345, -0.018081, -0.446181,
       -0.139854, -0.006102, -0.403974,  0.001501,  0.021966, -0.43595 ,
        0.144545, -0.006102, -0.403974,  0.002345, -0.066678, -0.445537,
       -0.129798, -0.059359, -0.40573 , -0.140946,  0.028435, -0.388389,
        0.145285,  0.028404, -0.388583,  0.132084, -0.059637, -0.406831,
        0.192732,  0.071757, -0.041727,  0.196616,  0.010706, -0.317276,
        0.191451,  0.076401, -0.039019,  0.19616 ,  0.008376,  0.29926 ,
        0.181463, -0.078195, -0.001712,  0.19209 , -0.118797, -0.286042,
        0.197869,  0.020274, -0.28742 ,  0.198237,  0.027142,  0.256007,
        0.19209 , -0.107019,  0.26609 ,  0.001707, 

In [50]:
y_train[0]

array([-1.90569937e-01,  2.16046330e-02, -2.42814086e-02, -1.83027416e-01,
       -5.75248152e-04,  3.04515243e-01, -1.83217198e-01,  6.07237145e-02,
       -3.82540114e-02, -1.83499277e-01, -1.13823311e-02, -3.08607608e-01,
       -1.74083784e-01, -6.93430454e-02, -3.85557255e-03, -1.81659818e-01,
       -9.26676616e-02,  2.62888491e-01, -1.77214965e-01,  5.37756905e-02,
        2.89658010e-01, -1.78229228e-01,  3.21672484e-02, -2.69728482e-01,
       -1.81090280e-01, -9.46186632e-02, -2.68231452e-01,  3.77157331e-03,
       -2.22427957e-02, -4.29328889e-01, -1.35522544e-01, -2.63939966e-02,
       -4.06195939e-01, -5.12617640e-04,  4.54471121e-03, -4.19039786e-01,
        1.32203668e-01, -2.92866174e-02, -4.10046130e-01,  1.58825703e-03,
       -6.69832081e-02, -4.21616077e-01, -1.27337530e-01, -6.67602643e-02,
       -4.04059321e-01, -1.36281550e-01, -8.13290011e-04, -3.94701898e-01,
        1.35797545e-01,  4.93134093e-03, -3.95223409e-01,  1.30829513e-01,
       -6.47926629e-02, -

In [59]:
delta = x_train[0] - y_train[0]
delta / x_train[0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


array([-1.34488608e-02,  6.98919507e-01,  4.18088800e-01,  4.40885132e-02,
        1.06867815e+00, -1.75607919e-02,  1.89750662e-02,  2.05197386e-01,
        1.96055403e-02,  4.39061063e-02,  2.06317309e+00,  2.73212973e-02,
        1.52128197e-02,  1.13203589e-01, -1.24683715e+00,  3.06306420e-02,
        1.34100846e-01,  1.30146190e-02,  8.43969764e-02, -9.79376122e-01,
       -1.32042623e-01,  7.73834098e-02, -5.86625650e-01,  6.15528433e-02,
        3.36697950e-02,  2.03526493e-01,  6.13992277e-02, -6.08346825e-01,
       -2.30175083e-01,  3.77696750e-02,  3.09712673e-02, -3.32546649e+00,
       -5.50020196e-03,  1.34151742e+00,  7.93102467e-01,  3.87893431e-02,
        8.53805503e-02, -3.79951121e+00, -1.50309931e-02,  3.22704891e-01,
       -4.57734276e-03,  5.36900921e-02,  1.89561453e-02, -1.24686473e-01,
        4.11771205e-03,  3.30938804e-02,  1.02860172e+00, -1.62540600e-02,
        6.53023690e-02,  8.26385688e-01, -1.70887788e-02,  9.49764489e-03,
       -8.64507413e-02, -